# MultiQuery

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama3-8b-8192"
)

In [4]:
from langchain.prompts import ChatPromptTemplate

# multi query: different perspective of one question
template = '''
You are an AI Language model. Your task is to generate five different versions of the given user question to retrieve relevant documents from a vector database. By generating multiple perspective on the user question, your goal is to help the user overcome some of the limitations of the distance based similarity search. Provide these alternative questions separated by newlines. Just answer the questions nothing else. Original question: {question}'''

prompt_multi_query = ChatPromptTemplate.from_template(template)

generate_queries = (
    prompt_multi_query
    | llm
    | (lambda x: x.content)
    | (lambda x: x.split("\n"))
)


In [5]:
generate_queries

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='\nYou are an AI Language model. Your task is to generate five different versions of the given user question to retrieve relevant documents from a vector database. By generating multiple perspective on the user question, your goal is to help the user overcome some of the limitations of the distance based similarity search. Provide these alternative questions separated by newlines. Just answer the questions nothing else. Original question: {question}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000022FBEFA3800>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000022FBEFC0290>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))
| RunnableLambda(...)
| 

In [6]:
generate_queries.invoke("How can I improve my learning?")

['How can I enhance my academic performance?',
 '',
 'What strategies can I use to optimize my learning?',
 '',
 'What are some effective methods for boosting my intellectual capacity?',
 '',
 'How can I develop a personalized approach to learning?',
 '',
 'What techniques can I employ to improve my knowledge retention and recall?']

In [7]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

# embedding
model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}

hf_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

c:\Program Files\Python312\Lib\importlib\__init__.py:90: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  return _bootstrap._gcd_import(name[level:], package, level)
C:\Users\acer\AppData\Roaming\Python\Python312\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
C:\Users\acer\AppData\Roaming\Python

In [8]:
hf_embeddings

HuggingFaceBgeEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='BAAI/bge-small-en', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={'normalize_embeddings': True}, query_instruction='Represent this question for searching relevant passages: ', embed_instruction='', show_progress=False)

In [9]:
len(hf_embeddings.embed_query("How can I improve my learning?"))

384

In [10]:
from langchain.document_loaders import TextLoader

loader = TextLoader("./agent.txt")
document = loader.load()

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
document_split = splitter.split_documents(document)

In [12]:
document_split

[Document(metadata={'source': './agent.txt'}, page_content='What are AI Agents?\nAn artificial intelligence (AI) agent is a software program that can interact with its environment, collect data, and use the data to perform self-determined tasks to meet predetermined goals. Humans set goals, but an AI agent independently chooses the best actions it needs to perform to achieve those goals. For example, consider a contact center AI agent that wants to resolves customer queries. The agent will automatically ask the customer different questions, look up information in internal documents, and respond with a solution. Based on the customer responses, it determines if it can resolve the query itself or pass it on to a human.\n\nWhat are the key principles that define AI agents?\nAll software autonomously completes different tasks as determined by the software developer. So, what makes AI or intelligent agents special?'),
 Document(metadata={'source': './agent.txt'}, page_content="What are the 

In [13]:
from langchain.vectorstores import FAISS

#!pip install faiss-cpu

vectorstore = FAISS.from_documents(
    documents=document_split,
    embedding=hf_embeddings,
)

In [14]:
retriever = vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000022F804E87D0>, search_kwargs={})

In [15]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """get unique union of retrieved content"""
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    unique_docs = list(set(flattened_docs))
    return [loads(doc) for doc in unique_docs]

question = "Ethical consideration in ai"
chain = generate_queries | retriever.map() | get_unique_union

In [16]:
docs = chain.invoke(question)
docs

C:\Users\acer\AppData\Local\Temp\ipykernel_18188\910890659.py:7: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  return [loads(doc) for doc in unique_docs]


[Document(metadata={'source': './agent.txt'}, page_content='What are AI Agents?\nAn artificial intelligence (AI) agent is a software program that can interact with its environment, collect data, and use the data to perform self-determined tasks to meet predetermined goals. Humans set goals, but an AI agent independently chooses the best actions it needs to perform to achieve those goals. For example, consider a contact center AI agent that wants to resolves customer queries. The agent will automatically ask the customer different questions, look up information in internal documents, and respond with a solution. Based on the customer responses, it determines if it can resolve the query itself or pass it on to a human.\n\nWhat are the key principles that define AI agents?\nAll software autonomously completes different tasks as determined by the software developer. So, what makes AI or intelligent agents special?'),
 Document(metadata={'source': './agent.txt'}, page_content='Ethical chall

In [17]:
from operator import itemgetter

data = [
    {"name": "Alice", "age": 10},
    {"name": "Bob", "age": 12},
    {"name": "Charlie", "age": 11},
]

names = list(map(itemgetter("name"), data))

print(names)

['Alice', 'Bob', 'Charlie']


In [18]:
from  operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

template = """Answer the following question: {question}:< context given > Context: {context} <context> just answer the question nothing else"""

prompt = ChatPromptTemplate.from_template(template)
final_rag_chain = (
    {"context": chain, "question": itemgetter("question")}
    | prompt
    | llm
    | (lambda x: x.content)
)

final_rag_chain.invoke({"question": question})

'Ethical considerations in AI agents include:\n\n* Unfair, biased, or inaccurate results\n* Data privacy concerns\n* Technical complexities (e.g. integrating machine learning libraries with software applications)\n* Limited compute resources for training and deploying deep learning AI agents\n* Need for human reviews to ensure customers receive helpful and fair responses from agents\n* Potential for AI agents to create and act on more tasks without human oversight'

# RAG Fusion
Reciprocal RAG Fusion

EXPLANATION:

**According to Question1**
Order of doc by relevance
1. Doc1
2. Doc2
3. Doc3
4. Doc4

**According to Q2**
1. Doc3
2. Doc1
3. Doc2
4. Doc4

**According to Q3**
1. Doc4
2. Doc1
3. Doc3
4. Doc2

Rank Positions

- Doc1:
 -- Question 1 rank: 1
 -- Question 2 rank: 2
 -- Question 3 rank: 2
- Doc2:
 -- Question 1 rank: 2
 -- Question 2 rank: 3
 -- Question 3 rank: 4
- Doc3:
 -- Question 1 rank: 3
 -- Question 2 rank: 1
 -- Question 3 rank: 3
- Doc4:
 -- Question 1 rank: 4
 -- Question 2 rank: 4
 -- Question 3 rank: 1

Reciprocal RAG Fusion

using k = 80 (arbitrary value)

RRF(Docn): sum[from i=1 to p where p = num of question](1 / (k + n))

##### Doc1
- Reciprocal Rank (Question1): 1 / (80 + 1) = 1 / 81
- Reciprocal Rank (Question2): 1 / (80 + 2) = 1 / 82
- Reciprocal Rank (Question3): 1 / (80 + 3) = 1 / 83
- RRF(Doc1): 1 / 81 + 1 / 82 + 1 / 83

Based on RRF score rank the document.

In [19]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=80):
    """Reciprocal rank fusion that takes multiple lists of ranked documents and parameter k to return a single list of ranked documents"""
    fused_scores = {}
    
    # iterate over each list of ranked document
    for docs in results:
        # iterate
        for rank, doc in enumerate(docs):
            # document to a string format to use as a key ( to json )
            doc_str = dumps(doc)
            # if the document is not in the fused scores, add it with score 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
                
            # retrieve the current score of the document
            previous_score = fused_scores[doc_str]
            # updat the score
            fused_scores[doc_str] = previous_score + 1 / (rank + k)
            
    # sort the fused scores by the score in desscending order to get the final ranked list
    reranked_results = [
        (loads(doc), score) for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]
    # return the final result
    return reranked_results
            

In [20]:
retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion

retrieval_chain_rag_fusion

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='\nYou are an AI Language model. Your task is to generate five different versions of the given user question to retrieve relevant documents from a vector database. By generating multiple perspective on the user question, your goal is to help the user overcome some of the limitations of the distance based similarity search. Provide these alternative questions separated by newlines. Just answer the questions nothing else. Original question: {question}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000022FBEFA3800>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000022FBEFC0290>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))
| RunnableLambda(...)
| 

In [21]:
docs = retrieval_chain_rag_fusion.invoke({"question": question})
docs

[(Document(metadata={'source': './agent.txt'}, page_content='Ethical challenges\nIn certain circumstances, deep learning models may produce unfair, biased, or inaccurate results. Applying safeguards, such as human reviews, ensures customers receive helpful and fair responses from the agents deployed. \n\nTechnical complexities \nImplementing advanced AI agents requires specialized experience and knowledge of machine learning technologies. Developers must be able to integrate machine learning libraries with software applications and train the agent with enterprise-specific data. \n\nLimited compute resources\nTraining and deploying deep learning AI agents requires substantial computing resources. When organizations implement these agents on-premise, they must invest in and maintain costly infrastructure that is not easily scalable.'),
  0.0875),
 (Document(metadata={'source': './agent.txt'}, page_content='Implement tasks\nWith sufficient data, the AI agent methodically implements the ta

In [22]:
from langchain_core.runnables import RunnablePassthrough

template = '''answer the following question: {question} given context: {context} only answer the question nothing else.'''

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion,
     "question": itemgetter("question")}
    | prompt
    | llm
    | (lambda x: x.content)
)

final_rag_chain.invoke({"question": question})

'Ethical considerations in AI given context:\n\n* In certain circumstances, deep learning models may produce unfair, biased, or inaccurate results. Applying safeguards, such as human reviews, ensures customers receive helpful and fair responses from the agents deployed.\n* Data privacy concerns: Developing and operating advanced AI agents requires acquiring, storing, and moving massive volumes of data. Organizations should be aware of data privacy requirements and employ necessary measures to improve data security posture.'

# Decomposition
make step by step question and answer the question.

In [23]:
from langchain.prompts import ChatPromptTemplate

template = """
You are a assistant that generates multiple sub-questions related to an input question. The goal is to break down the input into a set of sub-problems that can be solved independently. Generate three search queries related to: {question}. Response: just response the queries only """

prompt_decomposition = ChatPromptTemplate.from_template(template)

In [24]:
generate_queries_decomposition = (
    prompt_decomposition
    | llm
    | (lambda x: x.content)
    | (lambda x: x.split("\n"))
)
generate_queries_decomposition

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='\nYou are a assistant that generates multiple sub-questions related to an input question. The goal is to break down the input into a set of sub-problems that can be solved independently. Generate three search queries related to: {question}. Response: just response the queries only '), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000022FBEFA3800>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000022FBEFC0290>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))
| RunnableLambda(...)
| RunnableLambda(...)

In [25]:
question = "What is llm agent in ai"

In [26]:
generate_queries_decomposition.invoke(question)

['Here are three sub-questions:',
 '',
 '1. What is Large Language Model (LLM) in AI?',
 '2. How does an LLM Agent interact with its environment in AI?',
 '3. What are the applications of LLM Agents in Artificial Intelligence?']

In [27]:
generate_queries_decomposition.invoke("How can I improve my learning?")

['Here are three search queries related to "How can I improve my learning?":',
 '',
 '1. What are the most effective learning strategies for retaining information?',
 '2. How can I optimize my study routine for better comprehension and retention?',
 '3. What are some popular learning tools and resources that can help improve my understanding of complex topics?']

In [28]:
template = """
Here is the question you need to answer
\n---\n {question}\n---\n
Here is available background question and answer pairs:
\n---\n {qa}\n---\n
Here is other context relevant to the question:
\n---\n {context}\n---\n
Use the above context and any background question and answer pair to answer the question {question}"""

decomposition_prompt = ChatPromptTemplate.from_template(template)

In [29]:
decomposition_prompt.messages

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'qa', 'question'], input_types={}, partial_variables={}, template='\nHere is the question you need to answer\n\n---\n {question}\n---\n\nHere is available background question and answer pairs:\n\n---\n {qa}\n---\n\nHere is other context relevant to the question:\n\n---\n {context}\n---\n\nUse the above context and any background question and answer pair to answer the question {question}'), additional_kwargs={})]

In [30]:
from operator import itemgetter

In [31]:
def format_qa_pairs(questions, answers):
    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question{i}: {question}\nAnswer {i}: {answer}\n"
    # return the string
    return formatted_string.strip()


In [32]:
from langchain_core.output_parsers import StrOutputParser

In [33]:
q_a_pairs = ""
question = "What is an ai agent?"

questions = generate_queries_decomposition.invoke(question)

for q in questions:
    c = (
        {"context": itemgetter("question") | retriever,
         "question": itemgetter("question"),
         "qa": itemgetter("qa")}
        | decomposition_prompt
        | llm
        | StrOutputParser()
    )
    answer = c.invoke({"question": q, "qa": q_a_pairs})
    q_a_pair = format_qa_pairs(q, answer)
    q_a_pairs += "\n\n" + q_a_pair

In [34]:
answer

'Based on the provided context and background question and answer pairs, the characteristics and features of AI agents are:\n\n1. **Rational Agents**: AI agents make rational decisions based on their perceptions and data to produce optimal performance and results.\n2. **Autonomous**: AI agents operate independently, making decisions and taking actions without human intervention.\n3. **Informed Decision-Making**: AI agents use machine learning to gather and process massive amounts of real-time data, allowing for better predictions and informed decision-making.\n4. **Improved Customer Experience**: AI agents can personalize product recommendations, provide prompt responses, and innovate to improve customer engagement, conversion, and loyalty.\n5. **Implement Tasks**: AI agents methodically implement tasks at hand, removing them from the list and proceeding to the next one, and evaluating if they have achieved the designated goal.\n6. **Data-Driven**: AI agents collect data from their env

# answer individually

In [35]:
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [36]:
promt_rag = hub.pull("rlm/rag-prompt")

In [37]:
def retrieve_and_rag(question, prompt_rag, sub_question_generator_chain):
    """Retrieve and RAG"""
    sub_questions = sub_question_generator_chain.invoke(question)
    rag_results = []
    for q in sub_questions:
        retreived_doc = retriever.get_relevant_documents(q)
        answer = (prompt_rag | llm | StrOutputParser()).invoke({"question": q, "context": retreived_doc})
        
        rag_results.append(answer)
    return rag_results, sub_questions

In [38]:
answers, questions = retrieve_and_rag(question, promt_rag, generate_queries_decomposition)

C:\Users\acer\AppData\Local\Temp\ipykernel_18188\2762158091.py:6: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retreived_doc = retriever.get_relevant_documents(q)


In [39]:
def format_qa_pairs(questions, answers):
    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {question}\n\n Answer: {answer}\n\n"
    return formatted_string.strip()

In [40]:
context = format_qa_pairs(questions, answers)

In [41]:
context

'Question 1: Here are three search queries related to "What is an AI Agent?":\n\n Answer: An AI agent is a software program that interacts with its environment, collects data, and uses the data to perform self-determined tasks to meet predetermined goals. It makes rational decisions based on its perceptions and data to produce optimal performance and results.\n\nQuestion 2: \n\n Answer: The key components of AI agent architecture are not specified in the provided context.\n\nQuestion 3: 1. What are the characteristics of an AI agent?\n\n Answer: AI agents have the following characteristics: they are rational, making decisions based on data and perceptions to produce optimal performance and results, and they are able to sense their environment through physical or software interfaces. They collect data, analyze it to predict the best outcomes, and use the results to formulate their next actions.\n\nQuestion 4: 2. What are the types of AI agents (e.g. narrow, general, super)?\n\n Answer: 

In [42]:
template = """
Here is a set of qa pairs: {context}
Use these to synthesize an answer to the question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

rag = (
    prompt | llm | StrOutputParser()
)

In [43]:
rag

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nHere is a set of qa pairs: {context}\nUse these to synthesize an answer to the question: {question}\n'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000022FBEFA3800>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000022FBEFC0290>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()

In [44]:
rag.invoke({"question": question, "context": context})

'Based on the provided QA pairs, here\'s a synthesized answer to the question "What is an AI Agent?":\n\nAn AI agent is a software program that interacts with its environment to collect data, analyze it, and make rational decisions to achieve predetermined goals. It has the following characteristics: it is rational, senses its environment through physical or software interfaces, collects data, analyzes it to predict the best outcomes, and uses the results to formulate its next actions. There are three main types of AI agents: narrow, general, and super, each with different capabilities. An AI agent interacts with its environment by acquiring information, determining goals, and implementing tasks, using physical or software interfaces to sense its environment, collect data, and apply it to make informed decisions.'

## Decomposition: Answer recursively

In [45]:
hf_embeddings

HuggingFaceBgeEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='BAAI/bge-small-en', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={'normalize_embeddings': True}, query_instruction='Represent this question for searching relevant passages: ', embed_instruction='', show_progress=False)

In [46]:
questions = questions[2:]

In [47]:
questions

['1. What are the characteristics of an AI agent?',
 '2. What are the types of AI agents (e.g. narrow, general, super)?',
 '3. How does an AI agent interact with its environment?']

In [48]:
# prompt
template = """
Here is the question you need to answer:
\n----\n {question}\n --- \n
Here is any available background question and answer pairs:
\n --- \n {q_a_pairs} \n --- \n
Here is additional context relevant to the question:
\n -{context}- \n
Use the above context and any background question and answer pairs to answer : {question}
"""

In [49]:
decomposition_prompt_recursively = ChatPromptTemplate.from_template(template)
decomposition_prompt_recursively

ChatPromptTemplate(input_variables=['context', 'q_a_pairs', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'q_a_pairs', 'question'], input_types={}, partial_variables={}, template='\nHere is the question you need to answer:\n\n----\n {question}\n --- \n\nHere is any available background question and answer pairs:\n\n --- \n {q_a_pairs} \n --- \n\nHere is additional context relevant to the question:\n\n -{context}- \n\nUse the above context and any background question and answer pairs to answer : {question}\n'), additional_kwargs={})])

In [50]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

def format_qa_pair(question, answer):
    formatted_string = ""
    formatted_string += f"Question: {question}\n Answer: {answer}\n"
    return formatted_string.strip()

q_a_pairs = ""
for q in questions:
    rag_chain = (
        {"context": itemgetter("question") | retriever, "question": itemgetter("question"), "q_a_pairs": itemgetter("q_a_pairs")}
        | decomposition_prompt_recursively
        | llm
        | StrOutputParser()
    )
    
    answer = rag_chain.invoke({"question": q, "q_a_pairs": q_a_pairs})
    q_a_pair = format_qa_pair(q, answer)
    q_a_pairs = q_a_pairs + "\n-------------------------\n" + q_a_pair 


In [51]:
print(q_a_pairs)


-------------------------
Question: 1. What are the characteristics of an AI agent?
 Answer: Based on the provided context and background information, the characteristics of an AI agent are:

1. **Rational**: AI agents make rational decisions based on their perceptions and data to produce optimal performance and results.
2. **Autonomous**: AI agents can independently choose the best actions to perform to achieve predetermined goals, without human intervention.
3. **Goal-oriented**: AI agents have specific goals set by humans, and they use data to plan and perform tasks to achieve those goals.
4. **Perception**: AI agents sense their environment through physical or software interfaces, collecting data to make informed decisions.
5. **Data-driven**: AI agents analyze the collected data to predict the best outcomes that support predetermined goals and formulate the next action.
6. **Self-determined**: AI agents can determine the best course of action to achieve their goals, without human

# Step back
in decomposition we are generalizing the question but here we are step back question to understand it more.

In [70]:
from sentence_transformers import SentenceTransformer
load_dotenv()

embedding_path = os.environ.get("EMBEDDING_PATH")

model = SentenceTransformer(embedding_path)

In [71]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [72]:
 # few shot example for step back technique
 from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
 
 examples = [
     {
         "input": "Could the members of the Police perform lawful arrests?",
         "output": "What can the members of the police do?"
     },
     {
         "input": "John Doe was born in what country?",
         "output": "What is John Doe's history?"
     }
 ]

In [75]:
example_prompt = ChatPromptTemplate.from_messages([
    ("human","{input}"),
    ("ai","{output}")
])
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)
few_shot_prompt

FewShotChatMessagePromptTemplate(examples=[{'input': 'Could the members of the Police perform lawful arrests?', 'output': 'What can the members of the police do?'}, {'input': 'John Doe was born in what country?', 'output': "What is John Doe's history?"}], input_variables=[], input_types={}, partial_variables={}, example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['output'], input_types={}, partial_variables={}, template='{output}'), additional_kwargs={})]))

In [80]:
type(few_shot_prompt)

langchain_core.prompts.few_shot.FewShotChatMessagePromptTemplate

In [81]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """You are an expert at world knowledge. Your task is to step back and paraphrase the question to give more generic step back question, which is easier to answer. Here are a few examples: """),
        few_shot_prompt,
        ("user","{question}")
    ]
)
final_prompt

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert at world knowledge. Your task is to step back and paraphrase the question to give more generic step back question, which is easier to answer. Here are a few examples: '), additional_kwargs={}), FewShotChatMessagePromptTemplate(examples=[{'input': 'Could the members of the Police perform lawful arrests?', 'output': 'What can the members of the police do?'}, {'input': 'John Doe was born in what country?', 'output': "What is John Doe's history?"}], input_variables=[], input_types={}, partial_variables={}, example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), addit

In [82]:
generate_queries_step_back = final_prompt | llm | StrOutputParser()
question = "has llm used in twitter?"
generate_queries_step_back.invoke(question)

'Have large language models been used in a specific social media platform?'

In [83]:
response_prompt_template = """
You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant.
Otherwise, ignore them if they are not relevant to you.

# {normal_context}
# {step_back_context}
# Original Question: {question}
# Answer:
"""

response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

chain = (
    {
        # retrieve 
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        # retrieve using step back question
        "step_back_context": generate_queries_step_back | retriever,
        "question": lambda x: x["question"]
    }
    | response_prompt
    | llm
    | StrOutputParser()
)

In [84]:
chain

{
  normal_context: RunnableLambda(...)
                  | VectorStoreRetriever(tags=['FAISS', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000022F804E87D0>, search_kwargs={}),
  step_back_context: ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert at world knowledge. Your task is to step back and paraphrase the question to give more generic step back question, which is easier to answer. Here are a few examples: '), additional_kwargs={}), FewShotChatMessagePromptTemplate(examples=[{'input': 'Could the members of the Police perform lawful arrests?', 'output': 'What can the members of the police do?'}, {'input': 'John Doe was born in what country?', 'output': "What is John Doe's history?"}], input_variables=[], input_types={}, partial_variables={}, examp

In [88]:
print(chain.invoke({"question": "what is agent in llm"}))

I'm happy to help! Based on the provided context, an AI Agent is a software program that can interact with its environment, collect data, and use the data to perform self-determined tasks to meet predetermined goals. It's a type of artificial intelligence (AI) that can make rational decisions based on its perceptions and data to produce optimal performance and results.

In the context of Large Language Models (LLMs), an agent refers to a software program that can interact with the model to generate text, answer questions, or perform other tasks. In this sense, the agent acts as an intermediary between the user and the LLM, allowing the user to input queries or requests and receive responses or output.

The key principles that define AI agents are:

1. Autonomy: AI agents can operate independently, making decisions and taking actions on their own.
2. Rationality: AI agents make rational decisions based on their perceptions and data to produce optimal performance and results.
3. Sensing:

# HyDE

In [91]:
from langchain_core.prompts import ChatPromptTemplate

template = """
please write a scientific paper passage to answer the question.

Question: {question}
passage: 
"""

prompt_hyde = ChatPromptTemplate.from_template(template)

generate_docs_for_retrieval = (
    prompt_hyde | llm | (lambda x: x.content)
)

response = generate_docs_for_retrieval.invoke({"question": "how agent work in llm ai?"})

print(response)

Here is a scientific paper passage that answers the question "How agents work in LLM AI?":

**Title:** Understanding the Mechanisms of Agents in Large Language Models (LLMs)

**Abstract:** Large Language Models (LLMs) have revolutionized the field of natural language processing by enabling efficient and effective processing of vast amounts of linguistic data. A key component of LLMs is the agent-based architecture, which allows for the modeling of complex language phenomena through the interaction of multiple agents. In this passage, we delve into the mechanisms of agents in LLMs, exploring their role in language understanding and generation.

**Introduction:** LLMs are constructed by training neural networks on vast amounts of text data, which enables them to learn complex patterns and relationships between words, phrases, and sentences. At the heart of these models are agents, which are autonomous entities that interact with each other to generate language outputs. Each agent represe

In [92]:
retrieval_chain = generate_docs_for_retrieval | retriever
retrieved_doc = retrieval_chain.invoke({"question": "how agent work in llm ai?"})

In [93]:
retrieved_doc

[Document(metadata={'source': './agent.txt'}, page_content='Informed decision-making\nAdvanced intelligent agents use machine learning (ML) to gather and process massive amounts of real-time data. This allows business managers to make better predictions at pace when strategizing their next move. For example, you can use AI agents to analyze product demands in different market segments when running an ad campaign. \n\nImproved customer experience\nCustomers seek engaging and personalized experiences when interacting with businesses. Integrating AI agents allows businesses to personalize product recommendations, provide prompt responses, and innovate to improve customer engagement, conversion, and loyalty. \n\nWhat are the key components of AI agent architecture?\nAgents in artificial intelligence may operate in different environments to accomplish unique purposes. However, all functional agents share these components.'),
 Document(metadata={'source': './agent.txt'}, page_content='How do

In [94]:
template = """
Answer the following question based on this context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nAnswer the following question based on this context:\n{context}\nQuestion: {question}\n'), additional_kwargs={})])

In [96]:
rag_chain = (
    prompt | llm | StrOutputParser()
)

In [100]:
response = rag_chain.invoke({"context": retrieved_doc, "question": "How does agent work in llm artificial intelligence?"})

In [101]:
print(response)

Based on the provided context, it appears that the question is asking how an AI agent works in the context of LLM (Large Language Model) artificial intelligence.

According to the provided text, an AI agent works by simplifying and automating complex tasks. It follows a specific workflow when performing assigned tasks, which includes:

1. Determining goals: The AI agent receives a specific instruction or goal from the user and breaks it down into smaller actionable tasks.
2. Acquiring information: The agent extracts necessary information from various sources, such as the internet, to act on the planned tasks.

Additionally, the text mentions that AI agents use machine learning (ML) to gather and process massive amounts of real-time data, which allows them to make informed decisions and improve their performance over time.

It is important to note that the provided text does not specifically mention LLM artificial intelligence, but rather provides a general overview of how AI agents wor